# Visualization and Filtering with Lonboard
This notebook demonstrates how to load the intermediate filtered data and visualize it using Lonboard's ArcLayer.

## Step 1: Install Required Packages
Ensure you have the required packages installed.

In [ ]:
!pip install geopandas polars lonboard pyarrow numpy sidecar pyogrio anywidget ipywidgets

## Step 2: Import Libraries and Initialize Sidecar

In [12]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pyarrow as pa
import polars as pl
from IPython.display import display, Markdown
from sidecar import Sidecar
import lonboard as lb
from lonboard.experimental import ArcLayer
from lonboard.layer_extension import BrushingExtension

## Step 3: Load Filtered Data

In [13]:
filtered_data_path = './geo/viajes_distritos500k_od.parquet'
filtered_df = pl.read_parquet(filtered_data_path).to_pandas()
display(Markdown("### Filtered data loaded successfully."))
print(filtered_df.head(2))

### Filtered data loaded successfully.

      fecha  periodo    origen  destino distancia actividad_origen  \
0  20230101        1  38029_AM  0601509       >50  trabajo_estudio   
1  20230101       11  38029_AM  3501604       >50     no_frecuente   

  actividad_destino estudio_origen_posible estudio_destino_posible  \
0              casa                     no                      no   
1   trabajo_estudio                     no                      no   

   residencia  ... viajes_km partition_0        ID  origen_lat  origen_lng  \
0           6  ...  3596.229     2023-01  38029_AM   28.796568  -17.899411   
1          35  ...  1305.817     2023-01  38029_AM   28.796568  -17.899411   

  destino_right ID_right  destino_lat  destino_lng origen_right  
0      38029_AM  0601509    38.803288    -7.077529      0601509  
1      38029_AM  3501604    28.111166   -15.444507      3501604  

[2 rows x 24 columns]


## Step 4: Filter Out Rows Where 'origen' Equals 'destino'

In [16]:
#filtered_df = filtered_df.filter(filtered_df['origen'] != filtered_df['destino'])
#display(Markdown("### Filtered out rows where 'origen' equals 'destino'."))
#print(filtered_df.head())

### Filtered out rows where 'origen' equals 'destino'.

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


## Step 5: Create source and target scatterplot layers

In [14]:
# Create GeoDataFrames for source and target
source_gdf = gpd.GeoDataFrame(filtered_df, geometry=gpd.points_from_xy(filtered_df.origen_lng, filtered_df.origen_lat))
target_gdf = gpd.GeoDataFrame(filtered_df, geometry=gpd.points_from_xy(filtered_df.destino_lng, filtered_df.destino_lat))


In [15]:
# Define colors and radius
source_fill_colors = [0, 128, 0]
target_fill_colors = [128, 0, 0]
target_line_colors = [255, 0, 0]
brushing_radius = 20000
brushing_extension = BrushingExtension()

In [22]:
# Define source and target layers
source_layer = lb.ScatterplotLayer.from_geopandas(
    source_gdf,
    get_fill_color=source_fill_colors,
    radius_scale=3000,
    pickable=False,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
)

target_layer = lb.ScatterplotLayer.from_geopandas(
    target_gdf,
    get_fill_color=target_fill_colors,
    radius_scale=3000,
    pickable=False,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
)

target_ring_layer = lb.ScatterplotLayer.from_geopandas(
    target_gdf,
    get_line_color=target_line_colors,
    radius_scale=2000,
    pickable=False,
    stroked=True,
    filled=False,
    line_width_min_pixels=2,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
)

/opt/conda/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/opt/conda/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/opt/conda/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


## Step 5: Create ArcLayer for Visualization
Create the ArcLayer using the filtered data and visualize it using Lonboard.

In [23]:
value = np.ones(len(filtered_df))  # Use a constant value for simplicity
get_source_position = filtered_df[['origen_lng', 'origen_lat']].to_numpy()
get_target_position = filtered_df[['destino_lng', 'destino_lat']].to_numpy()
table = pa.table({'value': value})



In [24]:

arc_layer = ArcLayer(
    table=table,
    get_source_position=get_source_position,
    get_target_position=get_target_position,
    get_source_color=[0, 128, 0],
    get_target_color=[128, 0, 0],
    get_width=1,
    opacity=0.4,
    pickable=True,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
)


In [37]:

# Initialize the sidecar for the map
sc = Sidecar(title='Lonboard Map')


In [38]:
DarkMatter = 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'
# Configure Lonboard view
lonboard_map = lb.Map(
    layers=[source_layer, arc_layer, target_ring_layer]
    #,picking_radius=1
    , basemap_style = DarkMatter
    # initial_view_state={
    #     'latitude': filtered_df['origen_lat'].mean(),
    #     'longitude': filtered_df['origen_lng'].mean(),
    #     'zoom': 10,
    #     'bearing': 0,
    #     'pitch': 45,
    # },
    # tooltip={
    #     'html': 'Trip from {origen_lat}, {origen_lng} to {destino_lat}, {destino_lng}',
    #     'style': {'color': 'white'}
    # }
)


In [39]:

# Display the map in the sidecar
with sc:
    display(lonboard_map)